In [1]:
from __future__ import print_function

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import urllib.request, json
import os
import numpy as np
import tensorflow as tf

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
# Parameters
learning_rate = 0.0001
training_epochs = 50
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 128 # 2nd layer number of neurons
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

In [3]:
# tf Graph input
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [4]:
# Create model
def multilayer_perceptron(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1, name ='ReLU') 
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2, name ='ReLU') 
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [5]:
with tf.name_scope('Model'):
    pred = multilayer_perceptron(X)

# Define loss and optimizer

with tf.name_scope('loss_op'):
    # Minimize error using cross entropy
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        logits=pred, labels=Y))
    
with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_op)

    
with tf.name_scope('Accuracy'):
    # Accuracy
    accuracy = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(accuracy, "float"))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [6]:
# Initializing the variables
init = tf.global_variables_initializer()


# Create a summary to monitor cost tensor
# tf.summary.scalar("loss", loss_op)
training_summary_loss = tf.summary.scalar("training_loss", loss_op)
validation_summary_loss = tf.summary.scalar("validation_loss", loss_op)

# Create a summary to monitor accuracy tensor
# tf.summary.scalar("accuracy", accuracy)
training_summary = tf.summary.scalar("training_accuracy", accuracy)
validation_summary = tf.summary.scalar("validation_accuracy", accuracy)

# Merge all summaries into a single op
# merged_summary_op = tf.summary.merge_all()
performance_summaries_train = tf.summary.merge([training_summary_loss,training_summary])
performance_summaries_validate = tf.summary.merge([validation_summary_loss,validation_summary])

In [7]:
if not os.path.exists('summaries'):
    os.mkdir('summaries')
if not os.path.exists(os.path.join('summaries','first')):
    os.mkdir(os.path.join('summaries','first'))

config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.9 # making sure Tensorflow doesn't overflow the GPU

session = tf.InteractiveSession(config=config)

In [8]:
with tf.Session() as sess:
    
    sess.run(init)
    
    summary_writer = tf.summary.FileWriter(os.path.join('summaries','first'), graph=session.graph)
    
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)            
            
            _, c, summary = sess.run([optimizer, loss_op, performance_summaries_train],
                                     feed_dict={X: batch_x,Y: batch_y})
               
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch

        for i in range(total_batch):
            batch_x, batch_y = mnist.validation.next_batch(batch_size)

            summary_val = sess.run(performance_summaries_validate,
                                     feed_dict={X: batch_x,Y: batch_y})
    
            summary_writer.add_summary(summary_val, epoch * total_batch + i)
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost={:.3f}".format(avg_cost),
                 "Training Accuracy:", accuracy.eval({X: mnist.train.images, Y: mnist.train.labels}),
                 "validation Accuracy:", accuracy.eval({X: mnist.validation.images, Y: mnist.validation.labels}))
    print("Optimization Finished!")
    
    print("Accuracy:", accuracy.eval({X: mnist.test.images, Y: mnist.test.labels}))

Epoch: 0001 cost=522.410 Training Accuracy: 0.38936365 validation Accuracy: 0.4046
Epoch: 0002 cost=192.494 Training Accuracy: 0.5893091 validation Accuracy: 0.6064
Epoch: 0003 cost=115.938 Training Accuracy: 0.68914545 validation Accuracy: 0.7018
Epoch: 0004 cost=85.272 Training Accuracy: 0.74545455 validation Accuracy: 0.758
Epoch: 0005 cost=68.516 Training Accuracy: 0.77885455 validation Accuracy: 0.7872
Epoch: 0006 cost=57.755 Training Accuracy: 0.8037091 validation Accuracy: 0.8102
Epoch: 0007 cost=50.036 Training Accuracy: 0.8220182 validation Accuracy: 0.8274
Epoch: 0008 cost=44.096 Training Accuracy: 0.83647275 validation Accuracy: 0.8448
Epoch: 0009 cost=39.358 Training Accuracy: 0.84703636 validation Accuracy: 0.8514
Epoch: 0010 cost=35.554 Training Accuracy: 0.8581273 validation Accuracy: 0.8588
Epoch: 0011 cost=32.351 Training Accuracy: 0.8662 validation Accuracy: 0.8662
Epoch: 0012 cost=29.673 Training Accuracy: 0.8725273 validation Accuracy: 0.8704
Epoch: 0013 cost=27.358